# Capao Urban Rate

This notebook aims to estimate the current urban growth rate for the place: Vale do Capão, Palmeiras, BA, Brasil. By using Google Earth satellite images and analyzing RGB pixel data, we can determine the growth rate in areas featuring houses, roads, construction sites, or where forests have been cleared for humans uses. For this case study, we have chosen COPERNICUS satellite images.

In [ ]:
import ee
import geemap as geemap
import pandas as pd


from pprint import pprint
from utils.utils import mileseconds_to_date
from utils.features import get_coordinates
from utils.contants import (
  PROJECT, 
  GEO_POINT,
  GEO_PLACE,
  BANDS_COPERNICUS,
  BANDS_LANDSET_7,
)

ee.Authenticate()
ee.Initialize(project=PROJECT)
geemap.ee_initialize()

# Feature Colletion 

In [ ]:
# Pre process urban features
df = pd.read_csv('./data/urban_features.csv')
raw_urban = df['.geo'].tolist()
urban = [get_coordinates(i) for i in raw_urban]

urban_features_list = [
  ee.Feature(ee.Geometry.Point(urban[i][0], urban[i][1]), {'class': 1 }) for i in range(len(urban))
]

# Pre process vegetation features
df = pd.read_csv('./data/vegetation_feature.csv')
raw_vegetation = df['.geo'].tolist()
vegetation = [get_coordinates(i) for i in raw_vegetation]

vegetation_features_list = [
  ee.Feature(ee.Geometry.Point(vegetation[i][0], vegetation[i][1]), {'class': 0 }) for i in range(len(vegetation))
]

# Feature collections
urban_features = ee.FeatureCollection(urban_features_list, 'urban')
vegetation_features = ee.FeatureCollection(vegetation_features_list, 'vegetation')

feature = urban_features.merge(vegetation_features)

# feature.getInfo()

# Dataset

In [ ]:
geo_point = ee.Geometry.Point(GEO_POINT)
geo_place = ee.Geometry.Polygon(GEO_PLACE)

image = (
  ee.Image(
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
      .filterBounds(geo_point)
      .filterDate('2023-01-01', '2023-12-01')
      .sort('CLOUDY_PIXEL_PERCENTAGE')
      .first()
  )
)

label = 'class'

feature_collection = image.select(BANDS_COPERNICUS).sampleRegions(
  collection = feature,
  properties = [label],
  scale = 10,
  geometries = True
)
# feature_collection.getInfo()

image.getInfo()

# Randon Forest Model

In [ ]:
dataset = feature_collection.randomColumn()
training = dataset.filter('random <= 0.8')
validation = dataset.filter('random > 0.8')

cls = ee.Classifier.smileRandomForest(10).train(training, label, BANDS_COPERNICUS)

# store the classifier 
%store cls

train_accuracy = cls.confusionMatrix()
validation_sample = validation.classify(cls)
validation_accuracy = validation_sample.errorMatrix(label, 'classification')


# display('Results', cls.explain().getInfo())
# display('Confusion Matrix', train_accuracy.getInfo())
# display('Training error matrix', train_accuracy)
# display('Training overall accuracy', train_accuracy.accuracy())
# display('Validation error matrix', validation_accuracy)
# display('Validation accuracy', validation_accuracy.accuracy())



# Image Series
Using Google Earch Image Collection to extract timeseries images to representate the current urban progression 

- USGS Landsat 7 Collection 2 Tier 1 Raw Scenes. Image collection started from 1997
- USGS Landsat 8 Collection 2 Tier 1 Raw Scenes. Image collection started from 2013
- USGS Landsat 9 Collection 2 Tier 1 Raw Scenes. Image collection started from 2021

Images from 1997 and 1998 aren't available in the dataset

In [ ]:

image_series = {}

for year in range(1999, 2012):
    image = (ee.ImageCollection("LANDSAT/LE07/C02/T1")
        .filterBounds(geo_point)
        .filterDate(f'{year}-01-01', f'{year}-12-30')
        .sort('CLOUD_COVER')
        .first()
    )
    image_raw_date = image.date().getInfo().get('value')
    image_date = mileseconds_to_date(image_raw_date)
    image_series[year] = {'image': image, 'date': image_date}

for year in range(2013, 2020):
    image = (ee.ImageCollection("LANDSAT/LC08/C02/T1")
        .filterBounds(geo_point)
        .filterDate(f'{year}-01-01', f'{year}-12-30')
        .sort('CLOUD_COVER')
        .first()
    )
    image_raw_date = image.date().getInfo().get('value')
    image_date = mileseconds_to_date(image_raw_date)
    image_series[year] = {'image': image, 'date': image_date}

for year in range(2020, 2024):
    image = (ee.ImageCollection("LANDSAT/LC09/C02/T1")
        .filterBounds(geo_point)
        .filterDate(f'{year}-01-01', f'{year}-12-30')
        .sort('CLOUD_COVER')
        .first()
    )
    image_raw_date = image.date().getInfo().get('value')
    image_date = mileseconds_to_date(image_raw_date)
    image_series[year] = {'image': image, 'date': image_date}


# landcover = image_1997_1.clip(geo_place)
current_map = geemap.Map(center=[-12.609558240448216,-41.501150593949305], zoom=15)
current_map.addLayer(image_series[1999]['image'], {'bands': ['B1', 'B2', 'B3'], 'min': 0, 'max': 2000, "gamma": 3}, '1997')
current_map

# Classify Land Cover

In [ ]:
#TODO: Classify all images in the image series adding the urban cover area 
#TODO: Create a datase with the urban area amount per year
#TODO: Create a Regression model to predict the urban area amount for next 10 years using the model. 

# Map view

In [ ]:
capao_landcover = image.clip(geo_place).select(BANDS_LANDSET_7).classify(cls)

landcover_area = ee.Image.pixelArea().addBands(capao_landcover)

feature_areas = landcover_area.reduceRegions(
  reducer = ee.Reducer.sum().group({
    "groupField": 1,
    "groupName": "class"
  }),
  geometry=feature,
  scale=10,
  maxPixel=1e90
)

map = geemap.Map(center=[-12.609558240448216,-41.501150593949305], zoom=15)

class_vis = {
    'min': 0,
    'max': 10,
    'palette': [
        'green',
        'ffbb22',
        'ffff4c',
        'f096ff',
        'fa0000',
        'b4b4b4',
        'f0f0f0',
        '0064c8',
        '0096a0',
        '00cf75',
        'fae6a0',
    ],
}


# Adjust this value to increase or decrease the contrast
vis_params = {"bands": ['B2', 'B3', 'B4'],  min: 0, max: 2000, "gamma": 3}

# map.add_layer(image, vis_params, 'Polygon')
map.add_layer(capao_landcover, class_vis, 'Classification')

# Display the map
map
